In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
* data as a flat matrix,
* labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

#### Problem 1
Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor t using nn.l2_loss(t). The right amount of regularization should improve your validation / test accuracy.



In [9]:
batch_size = 128
h1_num_neurons = 1024

# Because we will be needing weights and biases for multiple layers, lets define functions to generate the same
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
#   weights = tf.Variable(
#     tf.truncated_normal([image_size * image_size, num_labels]))
#   biases = tf.Variable(tf.zeros([num_labels]))

  #start with hidden layer random initialization of variables and biases with a low values
  h1_weights = weight_variable([image_size * image_size, h1_num_neurons])
  h1_bias = bias_variable([h1_num_neurons])
  
  # Training computation.
  pre_activation_logits = tf.matmul(tf_train_dataset, h1_weights) + h1_bias

  # Introducing 1-hidden layer Activation function: relu
  # Hidden layer actiavation  
  h1_out = tf.nn.relu(pre_activation_logits)
    
  # New weight and biases as per the hidden layer output
  h1_wo = weight_variable([h1_num_neurons, num_labels])  
  h1_bo = bias_variable([num_labels])
    
  # Training computation.
  # Calculating y based on weights and biases from hidden layer and then applying softmax functions and cross entropy, 
  # to get probabilities and make predictions.
  # Below, final 'y' is calculted based on relu output. Then we apply softmax to get the probabilities and then apply
  # cross entropy to measure the distance(loss function) b/w softmax probabilities and actual 1-hot labels.  
  logits = tf.matmul(h1_out, h1_wo) + h1_bo
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  #L2 regularization
  regularizers = tf.nn.l2_loss(h1_weights) + tf.nn.l2_loss(h1_bias) + tf.nn.l2_loss(h1_wo) + tf.nn.l2_loss(h1_bo)
  # Add the regularization to the loss
  loss += 1e-5 * regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, h1_weights) + h1_bias), h1_wo) + h1_bo)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, h1_weights) + h1_bias), h1_wo) + h1_bo)

In [10]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.718123
Minibatch accuracy: 7.8%
Validation accuracy: 36.1%
Minibatch loss at step 500: 0.463788
Minibatch accuracy: 85.9%
Validation accuracy: 85.2%
Minibatch loss at step 1000: 0.452799
Minibatch accuracy: 89.1%
Validation accuracy: 86.4%
Minibatch loss at step 1500: 0.351192
Minibatch accuracy: 92.2%
Validation accuracy: 86.0%
Minibatch loss at step 2000: 0.303233
Minibatch accuracy: 93.0%
Validation accuracy: 87.1%
Minibatch loss at step 2500: 0.322834
Minibatch accuracy: 93.0%
Validation accuracy: 87.5%
Minibatch loss at step 3000: 0.473850
Minibatch accuracy: 85.2%
Validation accuracy: 88.0%
Test accuracy: 93.9%


#### Problem 2
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

In [11]:
batch_size = 128
h1_num_neurons = 1024

# Because we will be needing weights and biases for multiple layers, lets define functions to generate the same
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
#   weights = tf.Variable(
#     tf.truncated_normal([image_size * image_size, num_labels]))
#   biases = tf.Variable(tf.zeros([num_labels]))

  #start with hidden layer random initialization of variables and biases with a low values
  h1_weights = weight_variable([image_size * image_size, h1_num_neurons])
  h1_bias = bias_variable([h1_num_neurons])
  
  # Training computation.
  pre_activation_logits = tf.matmul(tf_train_dataset, h1_weights) + h1_bias

  # Introducing 1-hidden layer Activation function: relu
  # Hidden layer actiavation  
  h1_out = tf.nn.relu(pre_activation_logits)
    
  # New weight and biases as per the hidden layer output
  h1_wo = weight_variable([h1_num_neurons, num_labels])  
  h1_bo = bias_variable([num_labels])
    
  # Training computation.
  # Calculating y based on weights and biases from hidden layer and then applying softmax functions and cross entropy, 
  # to get prob bilities and make predictions.
  # Below, final 'y' is calculted based on relu output. Then we apply softmax to get the probabilities and then apply
  # cross entropy to measure the distance(loss function) b/w softmax probabilities and actual 1-hot labels.  
  logits = tf.matmul(h1_out, h1_wo) + h1_bo
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  #L2 regularization
  regularizers = tf.nn.l2_loss(h1_weights) + tf.nn.l2_loss(h1_bias) + tf.nn.l2_loss(h1_wo) + tf.nn.l2_loss(h1_bo)
  # Add the regularization to the loss
  loss += 1e-5 * regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, h1_weights) + h1_bias), h1_wo) + h1_bo)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, h1_weights) + h1_bias), h1_wo) + h1_bo)

In [14]:
num_steps = 10

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.433581
Minibatch accuracy: 9.4%
Validation accuracy: 26.1%
Minibatch loss at step 2: 17.137701
Minibatch accuracy: 14.1%
Validation accuracy: 41.4%
Minibatch loss at step 4: 10.812403
Minibatch accuracy: 48.4%
Validation accuracy: 45.9%
Minibatch loss at step 6: 4.582095
Minibatch accuracy: 57.0%
Validation accuracy: 68.7%
Minibatch loss at step 8: 3.586541
Minibatch accuracy: 62.5%
Validation accuracy: 66.7%
Test accuracy: 81.1%


In [47]:
x = np.array([[0, 3], [6, 2]])

In [49]:
index = np.argsort(x)

In [53]:
index

array([[0, 1],
       [1, 0]])

0